In [15]:
import numpy as np
import matplotlib.pyplot as plt

In [1]:
# import data from CSV data
import processData as pd

In [2]:
A = pd.getdata()

/home/chenyf1/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


array([[34.        ,  2.        ,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [31.        ,  0.        , 35.        , ...,  0.        ,
         1.        ,  0.        ],
       [33.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [33.        ,  1.50578826,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [46.        ,  1.51577159,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [38.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  0.        ]])

In [6]:
#proj = HP.hadamardProj_matrix(A,13,500)
#print(proj.shape)

NameError: name 'HP' is not defined

In [7]:
def generateHadamard (k):
    h = np.array([[1,1],[1,-1]])
    H = h
    for i in range(k)[1:]:
        H = np.kron(H,h)
    return H

In [8]:
#generate a vector 
x = A[:,1].reshape(-1,1)
print(x.shape)
print(x[:10])

(8192, 1)
[[ 2.]
 [ 0.]
 [ 1.]
 [ 2.]
 [ 6.]
 [ 3.]
 [ 3.]
 [ 2.]
 [10.]
 [ 3.]]


In [9]:
def result_Dx(x):
    length, _ = x.shape;
    d_vec = np.random.randint(2,size=(length,1))*2-1
    return d_vec*x

In [10]:
def chooseP(row,dimension):
    P = np.zeros(shape=(row,dimension))
    selector = np.random.randint(dimension-1,size=(row,1))+1
    for index in range(row):
        P[index,selector[index]-1] = 1
    return P,selector

In [11]:
count=0

In [12]:

# In the following algorithm,we want to track the number of how many non-zero elements in P. not only numbers, we want know 
# the positions in P,How do we obtain the non-zero elements' position
# If the elements in a row are all zero, we denote selector to zero
def iterator_hadamardProj(P,H,x,selector):
    dimension, _ = x.shape
    row,_ = P.shape
    if np.sum(selector==0) == row:
        print(np.sum(selector==0))
        print(P.shape)
        #count += 1
        results = np.zeros((row,1))
    elif np.sum(selector==0) ==row-1:
        #print(np.sum(selector==0))
        nonzero_index = np.argmax(selector)
        #print(P.shape)
        #print(nonzero_index,H.shape)
        #count += 1
        results = np.zeros((row,1))
        results[nonzero_index] = np.dot(H[selector[nonzero_index]-1],x)
    else:
        dimen_sub = int(dimension/2)
        P1 = P[:,:dimen_sub]
        P2 = P[:,dimen_sub:dimension]
        #Here we need to update the selector 
        selector1=(selector<dimen_sub)*selector
        selector2=(selector>=dimen_sub)*(selector-dimen_sub)
        H_sub = H[:dimen_sub,:dimen_sub]
        x_1=x[:dimen_sub]
        x_2 = x[dimen_sub:]
        #print(P1.shape,H_sub.shape,x_1.shape)
        #print(P2.shape,x_2.shape)
        res_add = iterator_hadamardProj(P1,H_sub,x_1+x_2,selector1)
        res_dec = iterator_hadamardProj(P2,H_sub,x_1-x_2,selector2)
        results = res_add + res_dec
    return results

In [13]:
def hadamardProj_vector(x,k,row):
    H = generateHadamard(k)
    #print(H.shape)
    Dx = result_Dx(x)
    P,selector= chooseP(row,2**k)# The selector contain the position imformation of nonzero elements 
    nonzero_elements = row
    #check=P.dot(np.dot(H,Dx))
    proj = iterator_hadamardProj(P,H,Dx,selector)
    return proj#-check

In [16]:
proj=hadamardProj_vector(x,13,500)

In [17]:
def hadamardProj_matrix(A,k,row):
    n,d = A.shape
    proj_M = np.zeros(shape=(row,d))
    for i in range(d):
        proj_M[:,i] = hadamardProj_vector(A[:,i].reshape(-1,1),k,row).reshape(-1,)
    return proj_M


In [18]:
print(proj.shape)

(500, 1)


In [19]:
proj_mat=hadamardProj_matrix(A,13,500)

In [20]:
H = generateHadamard(13)
#print(H.shape)
P,selector= chooseP(500,2**13)
proj=P.dot(np.dot(H,A))
print(proj.shape)

(500, 65)


In [22]:
proj.shape

(500, 65)